In [5]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import seaborn as sns 
import pandas as pd
import numpy as np
import bocd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import yfinance as yf

from machine_learning_finance import (analyze_trades, 
    plot_backtest_analysis, RangeTrainingWindowUtil,
    metrics_to_dataframe, TraderEnv, make_price_marker_from_boolean, add_marker, add_line)

import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
from datetime import datetime, timedelta

init_notebook_mode(connected=True)
windows = [300, 600, 900, 1500]

symbol = "SPY"
#file = f"../backtests/{symbol}-model-back-test.csv"
file = f"../backtests/{symbol}-expert-back-test.csv"
period = 365
pd.options.display.max_rows = None
inverse = None # set to anything to graph inverse longs

def plot_win_loss(file, metrics):
    ledger = pd.read_csv(file)
    # Set the style for the plots
    sns.set(style='whitegrid')

    # Create a bar plot for profit and loss stats
    profit_stats, loss_stats = metrics['profit_stats'], metrics['loss_stats']
    profit_loss_df = pd.DataFrame([profit_stats, loss_stats], columns=['min', 'max', 'mean', 'median', 'std'], index=['profit', 'loss'])

    plt.figure(figsize=(12, 6))
    ax = sns.barplot(data=profit_loss_df.transpose(), palette='muted')
    ax.set_title('Profit and Loss Statistics')
    ax.set_ylabel('Value')

    # Show the plot
    plt.show()


        
def analyze_and_graph_range(file, symbol, start, end, df):
    ledger = pd.read_csv(file)
    hist_df, test_df = create_train_test_windows(df, start=start, end=end)    
    test_df = calc_probabilties_without_lookahead(test_df, hist_df, hist_cutoff="2023-05-15")
    plot_backtest_analysis(test_df, ledger, inverse=symbol)
    return ledger, test_df
   
def analyze_and_graph(file, symbol, period, df):
    ledger = pd.read_csv(file)
    now = datetime.now()
    start = now - timedelta(days=365)
    start = start.strftime('%Y-%m-%d')
    end = now.strftime('%Y-%m-%d')
    training_window = RangeTrainingWindowUtil(df, start, end)
    env = TraderEnv(symbol, training_window.test_df, training_window.full_hist_df)
    
    
    fig = plot_backtest_analysis(env.orig_timeseries_with_analytics, ledger, inverse=inverse)

    # convert each change point column to price of the average in question
    analytics_df = env.orig_timeseries_with_analytics
    periods=[30, 60, 90]
    colors=["blue", "purple", "green"]
    for period, color in zip(periods, colors):        
        marker_col =  f"cp-loc-{period}"
        trend_col = f"trend-{period}"
        analytics_df = make_price_marker_from_boolean(analytics_df, f"change-point-{period}", f"trend-{period}", marker_col) 
        add_marker(fig, analytics_df, marker_col, f"change-point-{period}", "diamond", 8, color)
        add_line(fig, analytics_df, trend_col, f"moving-avg-{period}", color)

    analytics_df["expert"] = env.expert_actions
    analytics_df["expert_buy"] =  np.where(analytics_df["expert"] == 0, analytics_df["Close"], np.nan)
    analytics_df["expert_sell"] =  np.where(analytics_df["expert"] == 1, analytics_df["Close"], np.nan)
    add_marker(fig, analytics_df, "expert_buy", "expert_buy", "triangle-up", 8, "springgreen")
    add_marker(fig, analytics_df, "expert_sell", "expert_sell", "triangle-down", 8, "purple")

    fig.update_layout(title='Backtest Analysis', xaxis_title='Date', height=800)
    fig.show()

    
    fig = make_subplots(rows=3, cols=1)
    indexes=[1,2,3]
    for period, color, index in zip(periods, colors, indexes):        
        fig.add_trace(go.Bar(x=analytics_df.index, y=analytics_df[f'polynomial_derivative-{period}'], marker=dict(color=f'{color}', colorscale='Viridis'), name=f'Polynomial Derivative {period}'), row=index, col=1)

    fig.update_layout(title='Backtest Analysis', xaxis_title='Date', height=800)
    fig.show()

    return ledger, training_window.test_df, env, analytics_df

ticker_obj = yf.download(tickers=symbol)
df = pd.DataFrame(ticker_obj)


ledger, df2, env, analytics_df = analyze_and_graph(file, symbol, 365*4, df)    
#ledger, df2 = analyze_and_graph_range(file, symbol, "03-27-2020", "12-17-2021", df)

# Display metrics as text
metrics = analyze_trades(ledger, symbol, period)
metrics["file"] = file
metrics_df = metrics_to_dataframe(metrics) 
analytics_df
#df2[["moving_avg", "change_points", "rt", "Close"]]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,trend-30,trend-diff-30,change-point-30,polynomial_derivative-30,...,long_entry,long_exit,short_entry,short_exit,cp-loc-30,cp-loc-60,cp-loc-90,expert,expert_buy,expert_sell
Date,,,,,,,,,,,,,,,,,,,,,
2022-07-27,394.359985,402.880005,394.049988,401.040009,394.596313,82342100,383.713999,17.326009,False,1.108310,...,401.040009,NaN,NaN,NaN,NaN,NaN,NaN,0,401.040009,NaN
2022-07-28,401.890015,406.799988,398.149994,406.070007,399.545532,73966600,384.787333,21.282674,False,1.128053,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,406.070007,NaN
2022-07-29,407.579987,413.029999,406.769989,411.989990,405.370392,87003700,385.880332,26.109658,False,1.147796,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,411.989990,NaN
2022-08-01,409.149994,413.410004,408.399994,410.769989,404.170013,69997500,387.350999,23.418990,False,1.167540,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,410.769989,NaN
2022-08-02,409.119995,413.000000,406.820007,408.059998,401.503510,63435400,388.757666,19.302332,False,1.187283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,408.059998,NaN
2022-08-03,410.299988,415.679993,410.000000,414.450012,407.790863,67820600,390.070333,24.379679,False,1.207027,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,414.450012,NaN
2022-08-04,414.369995,415.089996,412.440002,414.170013,407.515350,45656600,391.396333,22.773681,False,1.226770,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,414.170013,NaN
2022-08-05,409.660004,414.149994,409.600006,413.470001,406.826630,56814900,392.576666,20.893335,False,1.246513,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,413.470001,NaN
2022-08-08,415.250000,417.619995,411.829987,412.989990,406.354309,53886100,393.340333,19.649657,False,1.266257,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,412.989990,NaN


In [ ]:
ledger